In [6]:
!pip install langchain-core langchain-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 30.9 MB/s eta 0:00:00


###**0. API Keys**

In [7]:
# Retrieve the API key from Colab's secrets
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
NEWS_API_KEY = userdata.get('news_api_key')
OPENWEATHER_API_KEY = userdata.get('openweather_api_key')

###**1. Define the Tools**

In [172]:
from langchain_core.tools import tool
import httpx
import urllib.parse

@tool
def fetch_and_summarize_news(arg: str = None) -> str:
    """Fetch the latest news articles and summarize them."""
    try:
        # Fetch news
        url = f"https://newsapi.org/v2/top-headlines?sources=bbc-news&apiKey={NEWS_API_KEY.strip()}"

        response = httpx.get(url)
        response.raise_for_status()
        articles = response.json().get("articles", [])

        if not articles:
            return "No news articles found."

        # Prepare articles for summarization
        sanitized_articles = []
        for article in articles:
            title = (article.get('title') or '').replace('\n', ' ').replace('\r', ' ')
            description = (article.get('description') or '').replace('\n', ' ').replace('\r', ' ')
            sanitized_articles.append(f"Title: {title}\nDescription: {description}")

        news_text = "\n\n".join(sanitized_articles)
        prompt = f"Summarize the following news articles:\n\n{news_text}. Please do not write the title. The summary should be in point for each news."

        # Generate the summary
        summary = llm.invoke(prompt)
        return summary
    except Exception as e:
        return f"Error: {e}"

@tool
def fetch_and_review_weather(city: str) -> str:
    """Fetch the weather information for a given city and generate a review."""
    city = city.strip()  # Ensure no leading/trailing spaces

    # Encode the city name for the URL
    encoded_city = urllib.parse.quote(city)
    url = f"https://api.openweathermap.org/data/2.5/weather?q={encoded_city}&appid={OPENWEATHER_API_KEY}&units=metric"

    try:
        response = httpx.get(url)
        response.raise_for_status()
        weather_data = response.json()

        # Extract weather details
        weather = weather_data['weather'][0]['main']
        temperature = weather_data['main']['temp']

        # Generate the review
        input_text = f"The current weather in {city} is {weather} with a temperature of {temperature}°C. As an expert in weather forecast analysis, please provide an appropriate weather review."
        input_text = ''.join(c for c in input_text if c.isprintable())

        # Generate the review using the language model
        review = llm.invoke(input_text)
        return review
    except Exception as e:
        return f"Error: {e}"


###**2. Bind Tools to LLM**

In [82]:
from langchain_openai import ChatOpenAI

In [83]:
llm = ChatOpenAI(model="gpt-4o-mini", openai_api_key=api_key)

In [84]:
# Bind all tools
llm_with_tools = llm.bind_tools([
    fetch_and_summarize_news,
    fetch_and_review_weather
])

###**3. Setup Conversation and Call the LLM**

In [85]:
from langchain_core.messages import HumanMessage, ToolMessage

In [173]:
messages = [
        HumanMessage("summarize news and weather of bengaluru")
]
llm_output = llm_with_tools.invoke(messages)
messages.append(llm_output)

###**4. Handle Tool Calls**

In [174]:
tool_mapping = {
    "fetch_and_summarize_news": fetch_and_summarize_news,
    "fetch_and_review_weather": fetch_and_review_weather,
}


In [175]:
llm_output.tool_calls

[{'name': 'fetch_and_summarize_news',
  'args': {'arg': 'Bengaluru'},
  'id': 'call_dxL0P5Q2QmjVPsWpNMdK9pGt',
  'type': 'tool_call'},
 {'name': 'fetch_and_review_weather',
  'args': {'city': 'Bengaluru'},
  'id': 'call_tgL2uOcpBiEe0lCarO4sZE6R',
  'type': 'tool_call'}]

In [176]:
for tool_call in llm_output.tool_calls:
    tool = tool_mapping[tool_call["name"].lower()]
    tool_output = tool.invoke(tool_call["args"])
    messages.append(ToolMessage(tool_output, tool_call_id=tool_call["id"]))

### **5. Get Final Answer from LLM**

In [177]:
response = llm_with_tools.invoke(messages)
print(response.content)

### News Summary for Bengaluru

1. **Healthcare Initiatives**: Sir Jackie Stewart is funding a new dementia blood test through his charity, inspired by his wife's condition.
2. **Legal Issues**: An 84-year-old British woman, featured in a Netflix documentary, has been charged with fraud in Singapore.
3. **Pension Updates**: Starting the new tax year, the state pension has increased by 4.1%, and carers can now work more hours without losing key benefits.
4. **Healthcare Criticism**: Resident doctors are facing criticism from health officials for late notifications about job placements.
5. **Celebrity News**: Meghan Markle's controversial public image could prove beneficial as she expands her lifestyle brand.
6. **Family's Hopes for Justice**: The mother of Jean Charles de Menezes hopes a new Disney drama will bring to light the truth about her son's fatal shooting by police.
7. **Automotive News**: UK Prime Minister announced relaxed measures for car companies to support the industry in